In [ ]:
# The way which the data frame is loaded is deprecated !
#
#

In [ ]:
version = 'v2'

In [14]:
import re
import os
import time
import numpy as np
import pandas as pd
import scipy.stats as stats
from scipy import sparse

import seaborn as sns
import matplotlib.pyplot  as plt

from sklearn import preprocessing, metrics
from sklearn.metrics import recall_score
from sklearn.preprocessing import OneHotEncoder

import logging
logging.config.fileConfig('logger.conf')
logger = logging.getLogger('simpleLogger')
#use -> logger.info('some log comment')

AttributeError: module 'logging' has no attribute 'config'

In [31]:
df = pd.read_pickle('features/' + version + '/data-exploration-df.pkl')
df_train_table = df[df.path == 'dataset/train_set/']
df_test_table = df[df.path == 'dataset/test_set/']

In [10]:
# BASELINE MODEL ! ! !
#
#

encoder = preprocessing.LabelEncoder()
valid_y = encoder.fit_transform(df_test_table.category.tolist())

letter_types = sorted(list(set(df.category.tolist())))

for i, v in enumerate(letter_types):
    if v == 'RL':
        rl_index = i

dummy_pred = []
for _ in range(0,len(valid_y)):
    dummy_pred.append(rl_index)

from sklearn.metrics import accuracy_score
accuracy_score(valid_y, dummy_pred)

0.38247011952191234

In [ ]:
# MY MODEL ! ! !
# 
#

In [16]:
df_cat_dummy = pd.DataFrame()
df_cat_dummy['file_type'] = np.zeros(len(df.category.unique()))
df_cat_dummy.index = df.category.unique()

In [17]:
def feature_prob(filter_key, filter_key_value, 
                 group_key='category', sort_key_value='category'):
    
    df_filtered = df[df[filter_key] == filter_key_value]
    df_grouped = df_filtered.groupby(group_key).count()

    use_dummy_df = True
    if use_dummy_df:
        #agrego categories que no estan 
        for i in range(0, len(df_cat_dummy)):
            item = df_cat_dummy.iloc[i]
            if item.name not in df_grouped.index:
                df_grouped = df_grouped.append(item)

    df_grouped.sort_values(by=[sort_key_value], inplace=True)
    
    total = df_grouped.file_type.sum()
    df_grouped['percentage'] = round(df_grouped.file_type / total * 100, 2)
    
    return df_grouped.percentage



In [27]:
def predict(item):
    
    prob = feature_prob(filter_key='file_type', filter_key_value=item.file_type)
    
    prob = prob + feature_prob(filter_key='enumeration_presence', filter_key_value=item.enumeration_presence)
    
    if item.enumeration_presence:
        prob = prob + feature_prob(filter_key='enumeration_repeated', filter_key_value=True)
    
    prob = prob + feature_prob(filter_key='sec_header_presence', filter_key_value=item.sec_header_presence)
    
    prob = prob + feature_prob(filter_key='cl_sentence_presence', filter_key_value=item.cl_sentence_presence)
    
    prob = prob + feature_prob(filter_key='response_presence', filter_key_value=item.response_presence)
    
    prob = prob + feature_prob(filter_key='most_freq_words_count', filter_key_value=item.most_freq_words_count)
    
    prob = prob + feature_prob(filter_key='less_freq_bgs_count', filter_key_value=item.less_freq_bgs_count)
    
    prob = pd.DataFrame(prob)
    max_prob = prob.max()
    pred = ''
    
    for i in range(0,len(prob)):
        if float(prob.iloc[i].percentage) == float(max_prob):
            pred = prob.iloc[i].name
    
    return pred

In [19]:
my_predictions = []
for i in range(0, len(df_test_table)):
    value_pred = predict(df_test_table.iloc[i])
    my_predictions.append(value_pred)

In [20]:
encoder = preprocessing.LabelEncoder()
encoder.fit(letter_types)
my_predictions_encoded = encoder.transform(my_predictions)

from sklearn.metrics import accuracy_score
accuracy = accuracy_score(valid_y, my_predictions_encoded)
print('accuracy', accuracy)
print()

items_recall = recall_score(valid_y, my_predictions_encoded, average=None)    
if len(letter_types) != len(items_recall):
    raise Exception('len(letter_types) != len(items_recall) ' + str(len(letter_types)) + ' != '+ str(len(items_recall)))
print(list(zip(letter_types,items_recall)))

accuracy 0.4641434262948207

[('ACAK', 0.0), ('AK', 0.0), ('AR', 0.0), ('CL', 0.26688102893890675), ('ER', 0.0), ('FL', 0.0), ('ND', 0.0), ('RL', 0.9973958333333334), ('TM', 0.0), ('UR', 0.0), ('UU', 0.0)]


In [ ]:
# MY MODEL ! ! !
# 
#

In [28]:
def predict_2(item):
    if item.response_presence:
        return 'RL'
    elif item.cl_sentence_presence:
        return 'CL'
    elif item.sec_header_presence == False:
        return 'UU'
    elif item.most_freq_words_count == 0:
        return 'UU'
    elif item.less_freq_bgs_count == 1:
        return 'UR'
    else:
        return 'RL'

In [32]:
my_predictions_2 = []
for i in range(0, len(df_test_table)):
    value_pred = predict_2(df_test_table.iloc[i])
    my_predictions_2.append(value_pred)

In [33]:
encoder = preprocessing.LabelEncoder()
encoder.fit(letter_types)
my_predictions_encoded = encoder.transform(my_predictions_2)

from sklearn.metrics import accuracy_score
accuracy = accuracy_score(valid_y, my_predictions_encoded)
print('accuracy', accuracy)
print()

items_recall = recall_score(valid_y, my_predictions_encoded, average=None)    
if len(letter_types) != len(items_recall):
    raise Exception('len(letter_types) != len(items_recall) ' + str(len(letter_types)) + ' != '+ str(len(items_recall)))
print(list(zip(letter_types,items_recall)))

accuracy 0.4641434262948207

[('ACAK', 0.0), ('AK', 0.0), ('AR', 0.0), ('CL', 0.24758842443729903), ('ER', 0.0), ('FL', 0.0), ('ND', 0.0), ('RL', 1.0), ('TM', 0.0), ('UR', 0.25), ('UU', 0.0375)]


In [ ]:
# MY MODEL ! ! !
# 
#

In [36]:
def predict_3(item):
    if item.response_presence:
        return 'RL'
    elif item.cl_sentence_presence:
        return 'CL'
    elif item.sec_header_presence == False:
        return 'UU'
    elif item.most_freq_words_count == 0:
        return 'UU'
    elif item.less_freq_bgs_count == 1:
        return 'UR'
    else:
        return predict(item)

In [34]:
my_predictions_3 = []
for i in range(0, len(df_test_table)):
    value_pred = predict_3(df_test_table.iloc[i])
    my_predictions_3.append(value_pred)

In [35]:
encoder = preprocessing.LabelEncoder()
encoder.fit(letter_types)
my_predictions_encoded = encoder.transform(my_predictions_3)

from sklearn.metrics import accuracy_score
accuracy = accuracy_score(valid_y, my_predictions_encoded)
print('accuracy', accuracy)
print()

items_recall = recall_score(valid_y, my_predictions_encoded, average=None)    
if len(letter_types) != len(items_recall):
    raise Exception('len(letter_types) != len(items_recall) ' + str(len(letter_types)) + ' != '+ str(len(items_recall)))
print(list(zip(letter_types,items_recall)))

accuracy 0.5268924302788844

[('ACAK', 0.0), ('AK', 0.0), ('AR', 0.0), ('CL', 0.4630225080385852), ('ER', 0.0), ('FL', 0.0), ('ND', 0.0), ('RL', 0.9947916666666666), ('TM', 0.0), ('UR', 0.0), ('UU', 0.0375)]
